In [3]:
import sys
#!{sys.executable} -m pip install selenium

#!{sys.executable} -m pip install beautifulsoup4
#import beautifulsoup4
import numpy as np # library to handle data in a vectorized manner
#from selenium import webdriver
import urllib.request
#import time
#from bs4 import BeautifulSoup
#from beautifulsoup4 import BeautifulSoup

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!{sys.executable} -m pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!{sys.executable} -m pip install requests
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#!{sys.executable} -m pip install matplotlib
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!{sys.executable} -m pip install sklearn
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!{sys.executable} -m pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from requests import get
from requests.exceptions import RequestException
from contextlib import closing

#!{sys.executable} -m pip install lxml
import lxml.html as lh


print('Libraries imported.')

Libraries imported.


<h3>Getting the Foursquare credentials</h3>

In [42]:
CLIENT_ID = '15OHIV1QRJPANM313MNOH144VYHGRIAWE1OXK4NEW1JP2UZC' # your Foursquare ID
CLIENT_SECRET = 'NNBZ1ZFDXGHADCHGWSO5B5LDS5UV2GBDJWZK1S3XILWQ4H4U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 15OHIV1QRJPANM313MNOH144VYHGRIAWE1OXK4NEW1JP2UZC
CLIENT_SECRET:NNBZ1ZFDXGHADCHGWSO5B5LDS5UV2GBDJWZK1S3XILWQ4H4U


<h3>Reading the downloaded dataset</h3>

In [26]:
df=pd.read_csv('delhi-neighborhood-data/delhi_dataSet.csv',index_col=0)
df.head()

,Borough,Neighborhood,latitude,longitude
0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
4,North West Delhi,Begum Pur,NaN,NaN


In [27]:
df=df.dropna(axis=0,how='any')
df.head()

,Borough,Neighborhood,latitude,longitude
0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
5,North West Delhi,Dhaka,39.031714,-90.261223


In [28]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 9 boroughs and 163 neighborhoods.


<h3>Generating the map of Delhi marked with all the neighborhoods</h3>

In [39]:
address = 'Central Delhi, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude =location.longitude
print('The geograpical coordinate of Delhi, India are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi, India are 28.69854835, 77.21939141568413.


In [106]:
map_dl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['latitude'], df['longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dl)  
    
map_dl

<h3>Getting the venue information for all the neighborhoods</h3>

In [43]:
radius=2000
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
map_dl.save('map_dl.html')


In [44]:
delhi_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Adarsh Nagar
Ashok Vihar
Azadpur
Bawana
Dhaka
Jahangirpuri
Karala
Keshav Puram
Kingsway Camp
Kohat Enclave
Model Town
Narela
Pitam Pura
Rani Bagh
Rithala
Shalimar Bagh
Shakti Nagar
Bara Hindu Rao
Chandni Chowk
Chawri Bazaar
Civil Lines
Dariba Kalan
Kamla Nagar
Kashmiri Gate
Kotwali
Lahori Gate
Nehru Vihar
Paharganj
Pratap Nagar
Pul Bangash
Roshanara Bagh
Sangam Vihar
Sant Nagar
Sarai Kale Khan
Sarai Rohilla
Shakti Nagar
Shastri Nagar
Timarpur
Wazirabad
Babarpur
Dilshad Garden
Kabir Nagar
Mandoli
Maujpur
Nand Nagri
Naveen Shahdara
Shahdara
Sonia Vihar
Shastri Park
Yamuna Vihar
Chandni Chowk
Daryaganj
Jhandewalan
Karol Bagh
Bara Hindu Rao
Paharganj
Barakhamba Road
Chanakyapuri
Connaught Place
Gautampuri
Gole Market
Golf Links
INA Colony
Khan Market
Pragati Maidan
Brij Puri
East Vinod Nagar
Gandhi Nagar
Geeta Colony
Jhilmil Colony
Krishna Nagar
Laxmi Nagar
Mayur Vihar
Pandav Nagar
Patparganj
Preet Vihar
Anand Vihar
Shakarpur
Shastri Nagar
Vasundhara Enclave
Vishwas Nagar
Vivek Vihar
Shahd

In [45]:
print(delhi_venues.shape)
delhi_venues.head()

(1086, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.614192,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
1,Ashok Vihar,28.699453,77.184826,SNAP FITNESS 24×7 GYM,28.700180,77.184582,Athletics & Sports
2,Ashok Vihar,28.699453,77.184826,Grand Plaza,28.696577,77.185245,Hotel Bar
3,Ashok Vihar,28.699453,77.184826,Moets Civil Lines,28.696452,77.184813,Chinese Restaurant
4,Ashok Vihar,28.699453,77.184826,Bakers Stop,28.700495,77.188716,Bakery


<h3>Counting the no of venues in each neighborhood</h3>

In [46]:
delhi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adarsh Nagar,1,1,1,1,1,1
Alaknanda,10,10,10,10,10,10
Anand Vihar,9,9,9,9,9,9
Ashok Vihar,7,7,7,7,7,7
Azadpur,3,3,3,3,3,3
Babarpur,3,3,3,3,3,3
Badarpur,3,3,3,3,3,3
Bali Nagar,6,6,6,6,6,6
Bara Hindu Rao,6,6,6,6,6,6


In [47]:
print('There are {} uniques categories.'.format(len(delhi_venues['Venue Category'].unique())))

There are 165 uniques categories.


<h3>Analyzing each neighborhood</h3>

In [48]:
t_onehot = pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
t_onehot['Neighborhood'] = delhi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [t_onehot.columns[-1]] + list(t_onehot.columns[:-1])
t_onehot = t_onehot[fixed_columns]



cols = list(t_onehot.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('Neighborhood')) #Remove b from list
#cols.pop(cols.index('x')) #Remove x from list
t_onehot = t_onehot[['Neighborhood']+cols] 
t_onehot.head()

,Neighborhood,Women's Store,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Café,Castle,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mosque,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,Nightlife Spot,North Indian Restaurant,Paper / Office Supplies Store,Park,Parking,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool Hall,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Rental Service,Restaurant,Road,Sake Bar,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ashok Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

<h3>Grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category</h3>

In [49]:
t_grouped = t_onehot.groupby('Neighborhood').mean().reset_index()
t_grouped

,Neighborhood,Women's Store,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Café,Castle,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mosque,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,Nightlife Spot,North Indian Restaurant,Paper / Office Supplies Store,Park,Parking,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool Hall,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Rental Service,Restaurant,Road,Sake Bar,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store
0,Adarsh Nagar,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0
1,Alaknanda,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.200000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.100000,0.00

<h3>Creating new dataframe for shopping mall data only</h3>

In [93]:
len(t_grouped[t_grouped["Shopping Mall"] > 0])


9

In [94]:
d_mall = t_grouped[["Neighborhood","Shopping Mall"]]
d_mall.head()

,Neighborhood,Shopping Mall
0,Adarsh Nagar,0.0
1,Alaknanda,0.0
2,Anand Vihar,0.0
3,Ashok Vihar,0.0
4,Azadpur,0.0


<h3>Clustering neighborhoods</h3>

In [105]:
kclusters = 3

d_clustering =d_mall.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(d_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)

In [96]:
d_merged = d_mall.copy()

# add clustering labels
d_merged["Cluster Labels"] = kmeans.labels_
d_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Adarsh Nagar,0.0,0
1,Alaknanda,0.0,0
2,Anand Vihar,0.0,0
3,Ashok Vihar,0.0,0
4,Azadpur,0.0,0


In [97]:
d_merged = d_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(d_merged.shape)
d_merged.head()

(146, 6)


,Neighborhood,Shopping Mall,Cluster Labels,Borough,latitude,longitude
0,Adarsh Nagar,0.0,0,North West Delhi,28.614192,77.071541
1,Alaknanda,0.0,0,South Delhi,28.529336,77.251632
2,Anand Vihar,0.0,0,East Delhi,28.625691,77.101941
3,Ashok Vihar,0.0,0,North West Delhi,28.699453,77.184826
4,Azadpur,0.0,0,North West Delhi,28.707657,77.175547


In [98]:
d_merged.sort_values(["Cluster Labels"], inplace=True)
d_merged

,Neighborhood,Shopping Mall,Cluster Labels,Borough,latitude,longitude
0,Adarsh Nagar,0.000000,0,North West Delhi,28.614192,77.071541
99,Preet Vihar,0.000000,0,East Delhi,28.637753,77.295175
98,Pratap Nagar,0.000000,0,North Delhi,28.666718,77.198897
97,Pragati Maidan,0.000000,0,New Delhi,28.623459,77.242512
96,Pitam Pura,0.000000,0,North West Delhi,28.703268,77.132250
95,Patparganj,0.000000,0,East Delhi,28.611592,77.290564
94,Patel Nagar,0.000000,0,West Delhi,28.659809,77.156957
93,Paschim Vihar,0.000000,0,West Delhi,28.669578,77.095956
92,Pandav Nagar,0.000000,0,East Delhi,28.650127,77.153278
91,Paharganj,0.000000,0,Central Delhi,28.641926,77.221750


<h3>Mapping the clustered data</h3>

In [99]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(d_merged['latitude'], d_merged['longitude'], d_merged['Neighborhood'], d_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [59]:
map_clusters.save('map_clusters.html')

<h3>Analyzing and listing each cluster</h3>

In [100]:
df2=d_merged.loc[d_merged['Cluster Labels'] == 0]
print(df.shape[0])
df2

163


,Neighborhood,Shopping Mall,Cluster Labels,Borough,latitude,longitude
0,Adarsh Nagar,0.0,0,North West Delhi,28.614192,77.071541
99,Preet Vihar,0.0,0,East Delhi,28.637753,77.295175
98,Pratap Nagar,0.0,0,North Delhi,28.666718,77.198897
97,Pragati Maidan,0.0,0,New Delhi,28.623459,77.242512
96,Pitam Pura,0.0,0,North West Delhi,28.703268,77.132250
95,Patparganj,0.0,0,East Delhi,28.611592,77.290564
94,Patel Nagar,0.0,0,West Delhi,28.659809,77.156957
93,Paschim Vihar,0.0,0,West Delhi,28.669578,77.095956
92,Pandav Nagar,0.0,0,East Delhi,28.650127,77.153278
91,Paharganj,0.0,0,Central Delhi,28.641926,77.221750


In [101]:
df2=d_merged.loc[d_merged['Cluster Labels'] == 1]
print(df.shape[0])
df2

163


,Neighborhood,Shopping Mall,Cluster Labels,Borough,latitude,longitude
119,Shakarpur,0.166667,1,East Delhi,28.629489,77.281061
77,Moti Nagar,0.125000,1,West Delhi,28.657859,77.142429
66,Laxmi Nagar,0.166667,1,East Delhi,28.630309,77.277563
7,Bali Nagar,0.166667,1,West Delhi,28.654138,77.128178
59,Kirti Nagar,0.200000,1,West Delhi,28.652368,77.149642


In [102]:
df2=d_merged.loc[d_merged['Cluster Labels'] == 2]
print(df.shape[0])
df2

163


,Neighborhood,Shopping Mall,Cluster Labels,Borough,latitude,longitude
115,Sarojini Nagar,0.090909,2,South Delhi,28.574158,77.195370
115,Sarojini Nagar,0.090909,2,South West Delhi,28.574158,77.195370
123,Shivaji Place,0.043478,2,West Delhi,28.651657,77.121703
57,Khirki Village,0.051282,2,South Delhi,28.529885,77.218077
103,Rama Krishna Puram,0.071429,2,South West Delhi,28.575561,77.172303


<h3>Observations</h3>

<p>Most of the shopping malls in Delhi are concentrated in the central and the southern regions of the state(as shown by the cluster 0).While the northern,north-western and western regions have very low amount of shopping malls as shown by the clusters 1 and 2.
    For a property developer who wants to find a place to build a shopping mall in Delhi,they can use this clustered data to see which area will be the most profitable due to less competition by other shopping malls.Shopping malls in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb areas still having very few shopping malls.
    Therefore this project recommends property developers to capitalize funds to open new shopping malls in areas of cluster 1 or 2 with little or no competition,and avoid areas of cluster 0 with high concentration of malls and oversupply of goods,thus resulting in intense competition.</p>